In [1]:
import json
from datetime import datetime
import uuid
import os

In [2]:
input_aws_file = "./dca394b2-da28-4218-b827-aade70011b23_SO-64051-20250608-20250614.json"
input_week_2 = "./week-2.json"
input_week_3 = "./week-3.json"

In [3]:
with open(input_aws_file, "r") as f:
    data = json.load(f)

with open(input_week_2, "r") as f:
    data_week_2 = json.load(f)

with open(input_week_3, "r") as f:
    data_week_3 = json.load(f)

In [7]:
tasks_list_week_2 = data_week_2["workitems"][0]["78c30344-5cfe-4744-b47f-7b51498ba6b9"]

In [9]:
len(tasks_list_week_2)

434

In [10]:
tasks_list_week_3 = data_week_3["workitems"][0]["78c30344-5cfe-4744-b47f-7b51498ba6b9"]

In [11]:
len(tasks_list_week_3)

652

In [99]:
# Separating two turn from three turn in week 2 delivery
three_turn_count = 0
two_turn_tasks = []
for task in tasks_list_week_2:
    if int(task["data"]["taskAnswers"][0]["dialogue_length"]) == 3:
        three_turn_count = three_turn_count + 1
    else:
        two_turn_tasks.append(task)

In [100]:
three_turn_count

7

In [36]:
two_turn_tasks[0]["data"]["taskAnswers"][0]["turnLevelOutput"][0]["prompt-turn"]["responses"]

[{'modelId': 'None',
  'responseText': '<< Impacts of climate change >>\n\nClimate change poses profound risks to both natural and human systems worldwide. Rising global temperatures, driven by human-induced emissions, are intensifying heatwaves and disrupting ecosystems, particularly in [urban regions] where vulnerable populations are at higher health risk. Uneven shifts in precipitation have led to increased rainfall in [tropical zones], causing frequent flooding and destruction of agricultural lands, while [arid regions] suffer from drought and water scarcity.\n\nThe melting of glaciers and polar ice caps continues to raise sea levels, endangering low-lying [coastal areas] and small island nations. This process is projected to accelerate, potentially displacing millions and eroding natural habitats. Meanwhile, warming oceans are triggering mass coral bleaching, leading to the breakdown of marine ecosystems and affecting the livelihoods of coastal communities reliant on fisheries and

In [55]:
for task in two_turn_tasks:
    prev_instr_dict = json.loads(task["data"]["taskAnswers"][0]["turnLevelOutput"][0]["instructions"])
    curr_instr_dict = json.loads(task["data"]["taskAnswers"][0]["turnLevelOutput"][1]["instructions"])
    metadata = set()
    change_details = []
    for instr in curr_instr_dict:
        instr_id = instr["instruction_id"]
        if instr_id not in prev_instr_dict:
            metadata.add("add")
            change_details.append({"change": "add", "instruction_id": instr_id})
        elif instr != prev_instr_dict[instr_id]:
            metadata.add("modify")
            change_details.append({"change": "modify", "instruction_id": instr_id})

    for instr_id in prev_instr_dict:
        if instr_id not in curr_instr_dict:
            metadata.add("remove")
            change_details.append({"change": "remove", "instruction_id": instr_id})

    task["data"]["taskAnswers"][0]["turnLevelOutput"][0]["instruction_change"] = []
    task["data"]["taskAnswers"][0]["turnLevelOutput"][1]["instruction_change"] = list(metadata)


In [71]:
new_workitems = []
for task in two_turn_tasks:
    task["data"]["taskAnswers"][0]["turnLevelOutput"][0]["prompt-turn"]["responses"][0]["modelId"] = "None"
    task["data"]["taskAnswers"][0]["turnLevelOutput"][1]["prompt-turn"]["responses"][0]["modelId"] = "None"
    new_workitem = {
        "workflow": "Verifiable Instruction Following - dialogue generation - 3P",
        "locale": "en-US",
        "metadata": {},
        "78c30344-5cfe-4744-b47f-7b51498ba6b9": [
            task
        ]
    }
    new_workitems.append(new_workitem)

In [80]:
final_workitems = []

for i in range(420):
    workitem = {}
    for key, value in new_workitems[i].items():
        workitem[key] = value
    for key, value in data['workitems'][i].items():
        workitem[key] = value
    final_workitems.append(workitem)

with open("final_workitems.json", "w") as f:
    json.dump(final_workitems, f)

In [81]:
with open("final_workitems.json", 'r') as f:
    final_workitems = json.load(f)
len(final_workitems)

420

In [72]:
len(new_workitems)

427

In [82]:
# Separating three turn from two turn in week 3 delivery
two_turn_count = 0
three_turn_tasks = []
for task in tasks_list_week_3:
    if int(task["data"]["taskAnswers"][0]["dialogue_length"]) == 2:
        two_turn_count = two_turn_count + 1
    else:
        three_turn_tasks.append(task)

In [83]:
for task in three_turn_tasks:
    prev_instr_dict = json.loads(task["data"]["taskAnswers"][0]["turnLevelOutput"][0]["instructions"])
    curr_instr_dict = json.loads(task["data"]["taskAnswers"][0]["turnLevelOutput"][1]["instructions"])
    metadata = set()
    change_details = []
    for instr in curr_instr_dict:
        instr_id = instr["instruction_id"]
        if instr_id not in prev_instr_dict:
            metadata.add("add")
            change_details.append({"change": "add", "instruction_id": instr_id})
        elif instr != prev_instr_dict[instr_id]:
            metadata.add("modify")
            change_details.append({"change": "modify", "instruction_id": instr_id})

    for instr_id in prev_instr_dict:
        if instr_id not in curr_instr_dict:
            metadata.add("remove")
            change_details.append({"change": "1remove", "instruction_id": instr_id})

    task["data"]["taskAnswers"][0]["turnLevelOutput"][0]["instruction_change"] = []
    task["data"]["taskAnswers"][0]["turnLevelOutput"][1]["instruction_change"] = list(metadata)


In [84]:
new_workitems = []
for task in three_turn_tasks:
    task["data"]["taskAnswers"][0]["turnLevelOutput"][0]["prompt-turn"]["responses"][0]["modelId"] = "None"
    task["data"]["taskAnswers"][0]["turnLevelOutput"][1]["prompt-turn"]["responses"][0]["modelId"] = "None"
    new_workitem = {
        "workflow": "Verifiable Instruction Following - dialogue generation - 3P",
        "locale": "en-US",
        "metadata": {},
        "78c30344-5cfe-4744-b47f-7b51498ba6b9": [
            task
        ]
    }
    new_workitems.append(new_workitem)

In [85]:
len(new_workitems)

551

In [88]:
final_workitems = []

for i in range(500):
    workitem = {}
    for key, value in new_workitems[i].items():
        workitem[key] = value
    for key, value in data['workitems'][i+500].items():
        workitem[key] = value
    final_workitems.append(workitem)

with open("final_workitems.json", "w") as f:
    json.dump(final_workitems, f)

In [91]:
# Separating two turn from three turn in week 3 delivery
three_turn_count = 0
two_turn_tasks_resubmission = []
for task in tasks_list_week_3:
    if int(task["data"]["taskAnswers"][0]["dialogue_length"]) == 3:
        three_turn_count = three_turn_count + 1
    else:
        two_turn_tasks_resubmission.append(task)

In [93]:
len(two_turn_tasks_resubmission)

101

In [94]:
for task in two_turn_tasks_resubmission:
    prev_instr_dict = json.loads(task["data"]["taskAnswers"][0]["turnLevelOutput"][0]["instructions"])
    curr_instr_dict = json.loads(task["data"]["taskAnswers"][0]["turnLevelOutput"][1]["instructions"])
    metadata = set()
    change_details = []
    for instr in curr_instr_dict:
        instr_id = instr["instruction_id"]
        if instr_id not in prev_instr_dict:
            metadata.add("add")
            change_details.append({"change": "add", "instruction_id": instr_id})
        elif instr != prev_instr_dict[instr_id]:
            metadata.add("modify")
            change_details.append({"change": "modify", "instruction_id": instr_id})

    for instr_id in prev_instr_dict:
        if instr_id not in curr_instr_dict:
            metadata.add("remove")
            change_details.append({"change": "1remove", "instruction_id": instr_id})

    task["data"]["taskAnswers"][0]["turnLevelOutput"][0]["instruction_change"] = []
    task["data"]["taskAnswers"][0]["turnLevelOutput"][1]["instruction_change"] = list(metadata)


In [95]:
new_workitems = []
for task in two_turn_tasks_resubmission:
    task["data"]["taskAnswers"][0]["turnLevelOutput"][0]["prompt-turn"]["responses"][0]["modelId"] = "None"
    task["data"]["taskAnswers"][0]["turnLevelOutput"][1]["prompt-turn"]["responses"][0]["modelId"] = "None"
    new_workitem = {
        "workflow": "Verifiable Instruction Following - dialogue generation - 3P",
        "locale": "en-US",
        "metadata": {},
        "78c30344-5cfe-4744-b47f-7b51498ba6b9": [
            task
        ]
    }
    new_workitems.append(new_workitem)

In [97]:
final_workitems = []

for i in range(80):
    workitem = {}
    for key, value in new_workitems[i].items():
        workitem[key] = value
    for key, value in data['workitems'][i+420].items():
        workitem[key] = value
    final_workitems.append(workitem)

with open("final_workitems.json", "w") as f:
    json.dump(final_workitems, f)